<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train9_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [33]:

train9 = pd.read_csv(r'/content/train_data9.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [34]:

train9['Target'] = train9['Target'].replace(['Others'],'Invalid')


In [35]:
train9['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [36]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train9_Xtfidf = tfidf_vect.fit_transform(train9['Sentence'])


In [38]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [39]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [40]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [41]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [42]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [43]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [44]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train9_Xtfidf,train9['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6780973451327433
                  precision    recall  f1-score   support

        Analysis       0.53      0.40      0.46       127
      Conclusion       0.76      0.46      0.57        48
           Facts       0.74      0.92      0.82       476
         Invalid       0.51      0.45      0.48       131
           Issue       0.51      0.44      0.47        52
Rule/Law/Holding       0.69      0.26      0.37        70

        accuracy                           0.68       904
       macro avg       0.62      0.49      0.53       904
    weighted avg       0.66      0.68      0.65       904



In [45]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [46]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3]
[0.9982862869255581, 0.9998887097899517, 0.9969887749364323, 0.9981618979260398, 0.9980350214986975, 0.9954912099699429, 0.9882748722131622, 0.9860930351423803, 0.9976729166163325, 0.9986371969073399, 0.9861154333169121, 0.9992633551560403, 0.9997883852392709, 0.9825584124259947, 0.9913830351857351, 0.9921545531366225, 0.9824119381667703, 0.9954123737350121, 0.9980889323216191, 0.9996103194734953, 0.9994103213525934, 0.9915286814258941, 0.9967018584484281, 0.9925542005485052, 0.9954452937401599, 0.9980862789784744, 0.9963479651238634, 0.994530960055934, 0.9993726541869734, 0.9926768397954673, 0.9885804638446272, 0.9999121515032106, 0.9845037533579707, 0.9928361503290031, 0.9914623833178718, 0.9970130202415852, 0.9994570626252551, 0.9899110665046481, 0.999389639718411, 0.9972475976914015]
[4, 5, 11, 12, 17, 18, 20, 21, 23, 24, 26, 29, 31, 33, 35, 36, 37, 39, 40, 45, 4

In [47]:
unlabel_1=unlabel_1.loc[[4, 5, 11, 12, 17, 18, 20, 21, 23, 24, 26, 29, 31, 33, 35, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 55, 56, 60, 62, 70, 72, 76, 79, 84, 87, 89, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts
12,1102,shown vine street drug storeâs supply narcotic...,Facts
17,2795,1951 ford came stop knocking several fence pos...,Facts


In [48]:
frame_1 = [train9,unlabel_1]
train9_1 = pd.concat(frame_1)
len(train9_1)

3516

In [49]:
x_train_1 = tfidf_vect.transform(train9_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train9_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6619318181818182
                  precision    recall  f1-score   support

        Analysis       0.47      0.42      0.44        98
      Conclusion       0.70      0.53      0.60        36
           Facts       0.74      0.89      0.81       361
         Invalid       0.51      0.49      0.50       112
           Issue       0.67      0.47      0.55        38
Rule/Law/Holding       0.56      0.17      0.26        59

        accuracy                           0.66       704
       macro avg       0.61      0.50      0.53       704
    weighted avg       0.64      0.66      0.64       704



In [50]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 5, 2, 2]
[0.9934157713035784, 0.9991494788084004, 0.9907145003488106, 0.9981259512043745, 0.9957714260281799, 0.9973504690585443, 0.99505331496181, 0.9999685900176868, 0.9835357937312894, 0.9872270680099915, 0.9959879974856214, 0.9943985241399407, 0.9957063181243941, 0.9939564380621625, 0.9997281472280116, 0.9801743564795339, 0.9966868621135465, 0.9928788869839974, 0.9992597262603402, 0.9998467515811618, 0.9990144823742997, 0.9978999364735318, 0.997833566559589, 0.9889156743982094, 0.9926838149751435, 0.9997119440321585, 0.9997324050538872, 0.9999151550279944, 0.9988934114353001, 0.99917102839097, 0.9899621950067626, 0.9973598965733951, 0.9992700653600773, 0.9851286057083614, 0.9940667917579794, 0.9980642708877719, 0.9921461006419107, 0.9991388646810153, 0.996228597431728, 0.9811039084706284, 0.9832035018561665, 0.9864848354814905]
[0, 2, 3, 4, 7, 8, 9, 12, 13, 14, 15, 

In [51]:
unlabel_2 = unlabel_2.loc[[0, 2, 3, 4, 7, 8, 9, 12, 13, 14, 15, 17, 20, 21, 29, 31, 32, 34, 36, 37, 39, 42, 49, 55, 58, 59, 63, 64, 69, 70, 71, 75, 79, 80, 83, 85, 86, 88, 91, 93, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [52]:
frame_2 = [train9_1,unlabel_2]
train9_2 = pd.concat(frame_2)
len(train9_2)

3558

In [53]:
x_train_2 = tfidf_vect.transform(train9_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train9_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6399583766909469
                  precision    recall  f1-score   support

        Analysis       0.45      0.36      0.40       152
      Conclusion       0.79      0.34      0.48        44
           Facts       0.72      0.90      0.80       497
         Invalid       0.45      0.42      0.44       152
           Issue       0.66      0.33      0.44        64
Rule/Law/Holding       0.47      0.29      0.36        52

        accuracy                           0.64       961
       macro avg       0.59      0.44      0.48       961
    weighted avg       0.62      0.64      0.62       961



In [54]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9873830848042588, 0.983868336387326, 0.9916693824755343, 0.9991678384529328, 0.9988339510364919, 0.9855374351096118, 0.9997313991821775, 0.9998254244122884, 0.9976274879629443, 0.9923039956919854, 0.9996263671508063, 0.997300249430878, 0.9807084915144086, 0.9886358998368214, 0.9951485779777033, 0.9982848284063456, 0.9851093482297262, 0.9927869926247341, 0.9993208190423553, 0.9981862773060277, 0.9914027061250434, 0.9857069339059319, 0.9965430986216746, 0.9921829938636172, 0.9973630947881598, 0.9925432721609274, 0.9991057106266245, 0.9949357029344448, 0.9998425589944498, 0.9908316490014804, 0.9996648838633572, 0.9989108180587384, 0.9996970834560525, 0.9958498136910898, 0.9814550245014788, 0.9996416283464987, 0.9894485361409598, 0.9971276064359966, 0.9951101345493245]
[0, 3, 4, 7, 8, 9, 12, 20, 24, 26, 27, 30, 32, 33, 35, 36, 41, 42, 43, 46, 47, 48, 51, 52, 54, 56, 63, 

In [55]:
unlabel_3=unlabel_3.loc[[0, 3, 4, 7, 8, 9, 12, 20, 24, 26, 27, 30, 32, 33, 35, 36, 41, 42, 43, 46, 47, 48, 51, 52, 54, 56, 63, 64, 65, 67, 68, 70, 72, 75, 79, 89, 91, 96, 97],:]
frame_3 = [train9_2,unlabel_3]
train9_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train9_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train9_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6666666666666666
                  precision    recall  f1-score   support

        Analysis       0.47      0.46      0.47        99
      Conclusion       0.72      0.50      0.59        46
           Facts       0.77      0.90      0.83       377
         Invalid       0.48      0.41      0.44       108
           Issue       0.56      0.47      0.51        43
Rule/Law/Holding       0.38      0.19      0.25        47

        accuracy                           0.67       720
       macro avg       0.56      0.49      0.51       720
    weighted avg       0.64      0.67      0.65       720



In [56]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 5, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 5, 2, 2]
[0.9997166160042732, 0.989266853667609, 0.9820628210164161, 0.9982486210969228, 0.9995709091964483, 0.9879211309460754, 0.9995203769670091, 0.9996835941691198, 0.993320822562224, 0.9969098525556737, 0.9962733725580641, 0.9983803476310468, 0.9986860855079067, 0.9950282284263546, 0.9996367741571833, 0.9991309872573035, 0.9992621692455558, 0.9957959771140389, 0.9997338983251248, 0.9962360812193594, 0.987214933054764, 0.9954167732231063, 0.9980812371795897, 0.9940779111793211, 0.9990167338978655, 0.996974639213401, 0.98291242616472, 0.9879237919022669, 0.9908299638080705, 0.9990556744382636, 0.9943120810894807, 0.9992278755521636, 0.9984905762853482, 0.9999688628591057, 0.9854749110549039, 0.9926175977391057, 0.9967433500746176, 0.9979691345353192, 0.9808510337234644, 0.9822565161025213, 0.9866298183424548, 0.9963194550334747]
[2, 4, 6, 7, 10, 13, 15, 18, 21, 22, 2

In [57]:
unlabel_4=unlabel_4.loc[[2, 4, 6, 7, 10, 13, 15, 18, 21, 22, 27, 28, 29, 30, 32, 34, 37, 38, 40, 46, 48, 49, 50, 55, 56, 58, 60, 61, 66, 67, 69, 70, 71, 75, 76, 82, 85, 87, 93, 94, 97, 99],:]

In [58]:
frame_4 = [train9_3,unlabel_4]
train9_4 = pd.concat(frame_4)
len(train9_4)
x_train_4 = tfidf_vect.transform(train9_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train9_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6581892166836215
                  precision    recall  f1-score   support

        Analysis       0.44      0.42      0.43       137
      Conclusion       0.59      0.51      0.55        45
           Facts       0.78      0.87      0.82       527
         Invalid       0.48      0.44      0.46       153
           Issue       0.42      0.32      0.36        57
Rule/Law/Holding       0.53      0.39      0.45        64

        accuracy                           0.66       983
       macro avg       0.54      0.49      0.51       983
    weighted avg       0.64      0.66      0.65       983



In [59]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[1, 2, 2, 3, 2, 1, 2, 0, 5, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 0, 0, 0]
[0.9886426431640405, 0.9989608023033518, 0.9958960724921877, 0.9941114021173972, 0.9836059314714207, 0.9937026716849556, 0.9974472266938726, 0.9897170904586117, 0.9948489316232823, 0.9895711744152554, 0.9853864827018741, 0.9959268058614232, 0.9993623453342907, 0.9996405942766057, 0.9942441001128594, 0.9861114251755754, 0.9849269500371729, 0.9998455200106829, 0.995884065784007, 0.999021191387897, 0.9994851978718741, 0.9859868588465773, 0.9978543689412583, 0.9983085787711139, 0.9980746356312363, 0.9929690524040534, 0.9931201719045941, 0.9999171243110908, 0.9981785877485344, 0.9838459873973809, 0.9995010527370789, 0.9987632502086737, 0.9889010886676665, 0.9930170975370581, 0.991884359646423, 0.98571232342021, 0.9962198181140695, 0.9957135607315065, 0.9897923800828821, 0.9894957173414664]
[0, 3, 7, 8, 9, 10, 12, 16, 18, 19, 20, 22, 23, 26, 27, 30, 32, 36, 46, 52, 53, 56,

In [60]:
unlabel_5 = unlabel_5.loc[[0, 3, 7, 8, 9, 10, 12, 16, 18, 19, 20, 22, 23, 26, 27, 30, 32, 36, 46, 52, 53, 56, 60, 63, 65, 67, 75, 82, 86, 91, 94, 95, 97, 100, 104, 106, 107, 110, 112, 114],:]

In [65]:
frame_5 = [train9_4,unlabel_5]
train9_5 = pd.concat(frame_5)
len(train9_5)
x_train_5 = tfidf_vect.transform(train9_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train9_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6750503018108652
                  precision    recall  f1-score   support

        Analysis       0.44      0.42      0.43       146
      Conclusion       0.64      0.43      0.51        54
           Facts       0.81      0.87      0.84       522
         Invalid       0.50      0.56      0.53       141
           Issue       0.59      0.51      0.55        59
Rule/Law/Holding       0.48      0.29      0.36        72

        accuracy                           0.68       994
       macro avg       0.58      0.51      0.54       994
    weighted avg       0.66      0.68      0.67       994



In [66]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6794625719769674
